In [2]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv('plazas_prueba.csv')

# Mostrar las primeras filas del DataFrame
print(df.head())

        Nombres Apellido Paterno Apellido Materno                       CCT  \
0       Ana Luz           Suárez             Luna          Maria Luisa Ross   
1  Luis Roberto          Ramírez          Jiménez      Enrique de Olavarr�a   
2       Ana Luz           Suárez             Luna             Independencia   
3  Luis Roberto          Ramírez          Jiménez  Maestro Ernesto Alconedo   
4       Ana Luz           Suárez             Luna              Luis Cabrera   

     LATITUD   LONGITUD  
0  19.375208 -99.230397  
1  19.376023 -99.188159  
2  19.297861 -99.649142  
3  19.381043 -99.163434  
4  19.371972 -99.159487  


In [3]:
import requests

# FUNCIÓN PARA HACER SOLICITUDES A LA API MAPBOX ISOCHRONES
def mapbox_isochrone(base_url, routing_profile, lon, lat, mins=None, meters=None, access_token=None):
    """
    Realiza una solicitud a la API Mapbox Isochrones y devuelve un GeoDataFrame con los vértices de la isócrona.
    """
    # Construye la URL de la solicitud
    if meters is not None:
        query = f"?contours_meters={meters}"
    elif mins is not None:
        query = f"?contours_minutes={mins}"
    else:
        raise ValueError("Debes especificar 'mins' o 'meters'.")
    
    url = f"{base_url}{routing_profile}/{lon},{lat}{query}&polygons=true&denoise=1&access_token={access_token}"

    # Realiza la solicitud a la API
    response = requests.get(url)
    response.raise_for_status()  # Lanza un error si la solicitud falla
    
    # Procesa los datos recibidos
    geom = response.json()["features"][0]["geometry"]["coordinates"][0]

    # Convierte las coordenadas a un DataFrame
    coords = pd.DataFrame(geom, columns=["lon", "lat"])
    return coords

In [4]:
# PRUEBA DE LA FUNCIÓN
test_coords = mapbox_isochrone(
    base_url="https://api.mapbox.com/isochrone/v1/mapbox/",
    routing_profile="walking",
    lon=-99.184823289979, lat=19.33905855077223,
    meters=500,
    access_token="pk.eyJ1IjoiY2FybG9hZDEzMTciLCJhIjoiY200aXAzaG9lMDRrcDJyb3BlMWs5bjZxbyJ9.cMOjch9URfW3X133K7MDQw"
)

test_coords

,lon,lat
0,-99.185823,19.341088
1,-99.186794,19.341029
2,-99.187358,19.339593
3,-99.188514,19.339059
4,-99.189093,19.338059
5,-99.187823,19.336529
6,-99.185823,19.335395
7,-99.184823,19.335892
8,-99.183823,19.335570
9,-99.182823,19.335835


In [5]:
import time
from shapely.geometry import Polygon

dataframe_ciclo = df  # Asignar el DataFrame que contiene los centros de trabajo

# CICLO PARA OBTENER ISÓCRONAS DE TODOS LOS CENTROS DE TRABAJO
isochrone_list = []  # Lista para guardar las isócronas
times = []  # Para estimar el tiempo de ejecución

for idx, row in dataframe_ciclo.iterrows():
    start_time = time.time()
    print(f"Obteniendo isócrona para la escuela {idx + 1} de {len(dataframe_ciclo)}...")

    try:
        # Obtener isócrona
        coords = mapbox_isochrone(
            base_url="https://api.mapbox.com/isochrone/v1/mapbox/",
            #routing_profile="driving-traffic",
            routing_profile="walking",
            lon=row["LONGITUD"], lat=row["LATITUD"],
            #mins=5,  # Tiempo de conducción
            meters=5000,  # Distancia
            access_token="pk.eyJ1IjoiY2FybG9hZDEzMTciLCJhIjoiY200aXAzaG9lMDRrcDJyb3BlMWs5bjZxbyJ9.cMOjch9URfW3X133K7MDQw"
        )
        # Convertir a polígono y añadir a la lista
        polygon = Polygon(coords.values)
        isochrone_list.append({"CCT": row["CCT"], "geometry": polygon})
    except Exception as e:
        print(f"Error al obtener la isócrona para la escuela {idx + 1}: {e}")
        continue

    # Calcular tiempo restante
    elapsed_time = time.time() - start_time
    times.append(elapsed_time)
    avg_time = sum(times) / len(times)
    remaining_time = avg_time * (len(dataframe_ciclo) - (idx + 1)) / 60
    print(f"Tiempo estimado restante: {remaining_time:.2f} minutos")

Obteniendo isócrona para la escuela 1 de 5...
Tiempo estimado restante: 0.02 minutos
Obteniendo isócrona para la escuela 2 de 5...
Tiempo estimado restante: 0.02 minutos
Obteniendo isócrona para la escuela 3 de 5...
Tiempo estimado restante: 0.01 minutos
Obteniendo isócrona para la escuela 4 de 5...
Tiempo estimado restante: 0.01 minutos
Obteniendo isócrona para la escuela 5 de 5...
Tiempo estimado restante: 0.00 minutos


In [6]:
import geopandas as gpd

# CONVERTIR A UN GEODATAFRAME
isochrones_gdf = gpd.GeoDataFrame(isochrone_list, crs="EPSG:4326")
#isochrones_gdf = gpd.GeoDataFrame(isochrone_list, crs="4326")

In [7]:
isochrones_gdf.head()

,CCT,geometry
0,Maria Luisa Ross,"POLYGON ((-99.2144 19.39523, -99.21508 19.3952..."
1,Enrique de Olavarr�a,"POLYGON ((-99.17816 19.41912, -99.17916 19.419..."
2,Independencia,"POLYGON ((-99.64114 19.33347, -99.64214 19.333..."
3,Maestro Ernesto Alconedo,"POLYGON ((-99.16243 19.42426, -99.16343 19.424..."
4,Luis Cabrera,"POLYGON ((-99.15449 19.41659, -99.15549 19.416..."


In [8]:
# UNIR RESULTADOS CON LA BASE ORIGINAL
escuelas_gdf = gpd.GeoDataFrame(dataframe_ciclo, geometry=None)
escuelas_gdf["geometry"] = None

for idx, row in escuelas_gdf.iterrows():
    match = isochrones_gdf[isochrones_gdf["CCT"] == row["CCT"]]
    if not match.empty:
        escuelas_gdf.at[idx, "geometry"] = match["geometry"].values[0]

/tmp/ipykernel_145848/2231656122.py:3: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  escuelas_gdf["geometry"] = None


In [13]:
escuelas_gdf

,Nombres,Apellido Paterno,Apellido Materno,CCT,LATITUD,LONGITUD,geometry
0,Ana Luz,Suárez,Luna,Maria Luisa Ross,19.375208,-99.230397,"POLYGON ((-99.2144 19.39523, -99.21508 19.3952..."
1,Luis Roberto,Ramírez,Jiménez,Enrique de Olavarr�a,19.376023,-99.188159,"POLYGON ((-99.17816 19.41912, -99.17916 19.419..."
2,Ana Luz,Suárez,Luna,Independencia,19.297861,-99.649142,"POLYGON ((-99.64114 19.33347, -99.64214 19.333..."
3,Luis Roberto,Ramírez,Jiménez,Maestro Ernesto Alconedo,19.381043,-99.163434,"POLYGON ((-99.16243 19.42426, -99.16343 19.424..."
4,Ana Luz,Suárez,Luna,Luis Cabrera,19.371972,-99.159487,"POLYGON ((-99.15449 19.41659, -99.15549 19.416..."


In [10]:
escuelas_gdf.dtypes

Nombres               object
Apellido Paterno      object
Apellido Materno      object
CCT                   object
LATITUD              float64
LONGITUD             float64
geometry            geometry
dtype: object

In [11]:
#escuelas_gdf.explore()

In [14]:
#AÑADIMOS UNA NUEVA COLUMNA CON EL NOMBRE COMPLETO DE LAS PERSONAS
escuelas_gdf['Nombre completo'] = escuelas_gdf.apply(lambda row: f"{row['Nombres']} {row['Apellido Paterno']} {row['Apellido Materno']}", axis=1)

# Mostrar el DataFrame
print(escuelas_gdf)

        Nombres Apellido Paterno Apellido Materno                       CCT  \
0       Ana Luz           Suárez             Luna          Maria Luisa Ross   
1  Luis Roberto          Ramírez          Jiménez      Enrique de Olavarr�a   
2       Ana Luz           Suárez             Luna             Independencia   
3  Luis Roberto          Ramírez          Jiménez  Maestro Ernesto Alconedo   
4       Ana Luz           Suárez             Luna              Luis Cabrera   

     LATITUD   LONGITUD                                           geometry  \
0  19.375208 -99.230397  POLYGON ((-99.2144 19.39523, -99.21508 19.3952...   
1  19.376023 -99.188159  POLYGON ((-99.17816 19.41912, -99.17916 19.419...   
2  19.297861 -99.649142  POLYGON ((-99.64114 19.33347, -99.64214 19.333...   
3  19.381043 -99.163434  POLYGON ((-99.16243 19.42426, -99.16343 19.424...   
4  19.371972 -99.159487  POLYGON ((-99.15449 19.41659, -99.15549 19.416...   

                  Nombre completo  
0           Ana Luz 

In [15]:
import folium

# VISUALIZAR RESULTADOS CON FOLIUM
m = folium.Map(location=[19.33905855077223, -99.184823289979], zoom_start=12)

for _, row in escuelas_gdf.iterrows():
    if row["geometry"] is not None:
        folium.Polygon(
            locations=[(p[1], p[0]) for p in list(row["geometry"].exterior.coords)],
            color="blue",
            weight=1, fill_color="blue",
            fill_opacity=0.5,
            fill=True
        ).add_to(m)
        # Añadir un punto al mapa
        folium.Marker(
            location=[row['LATITUD'], row['LONGITUD']],  # Coordenadas del punto
            popup=f"Persona: {row['Nombre completo']}, CCT: {row['CCT']}",  # Texto emergente al hacer clic en el marcador
            icon=folium.Icon(icon='info-sign')  # Icono del marcador
        ).add_to(m)

In [16]:
# Guardar resultados
isochrones_gdf.to_file("primarias.shp")
escuelas_gdf.to_file("escuelas_con_isocronas.shp")

/tmp/ipykernel_145848/1695719795.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  escuelas_gdf.to_file("escuelas_con_isocronas.shp")
/home/misael/.pyenv/versions/entorno_python_3.12.3/lib/python3.12/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
/home/misael/.pyenv/versions/entorno_python_3.12.3/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Apellido Paterno' to 'Apellido P'
  ogr_write(
/home/misael/.pyenv/versions/entorno_python_3.12.3/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Apellido Materno' to 'Apellido M'
  ogr_write(
/home/misael/.pyenv/versions/entorno_python_3.12.3/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Nombre 

In [17]:
# Mostrar el mapa interactivo
m.save("isocronas_escuelas.html")

En esta parte se programa un codigo para extraer de forma eficiente todas los centros de trabajo de una misma persona.

In [18]:
# Crear un diccionario para almacenar los DataFrames
dataframes = {}

# Agrupar por 'Nombre completo' y crear un DataFrame para cada grupo
for name, group in escuelas_gdf.groupby('Nombre completo'):
    dataframes[name] = group

# Mostrar los DataFrames creados
for name, df_group in dataframes.items():
    print(f"DataFrame para {name}:")
    print(df_group)
    print("\n")

DataFrame para Ana Luz  Suárez  Luna:
   Nombres Apellido Paterno Apellido Materno               CCT    LATITUD  \
0  Ana Luz           Suárez             Luna  Maria Luisa Ross  19.375208   
2  Ana Luz           Suárez             Luna     Independencia  19.297861   
4  Ana Luz           Suárez             Luna      Luis Cabrera  19.371972   

    LONGITUD                                           geometry  \
0 -99.230397  POLYGON ((-99.2144 19.39523, -99.21508 19.3952...   
2 -99.649142  POLYGON ((-99.64114 19.33347, -99.64214 19.333...   
4 -99.159487  POLYGON ((-99.15449 19.41659, -99.15549 19.416...   

         Nombre completo  
0  Ana Luz  Suárez  Luna  
2  Ana Luz  Suárez  Luna  
4  Ana Luz  Suárez  Luna  


DataFrame para Luis Roberto  Ramírez  Jiménez:
        Nombres Apellido Paterno Apellido Materno                       CCT  \
1  Luis Roberto          Ramírez          Jiménez      Enrique de Olavarr�a   
3  Luis Roberto          Ramírez          Jiménez  Maestro Ernesto Al

In [ ]:
from shapely.geometry import Point
from itertools import combinations

# Crear un DataFrame vacío con las columnas deseadas
df_out = pd.DataFrame(columns=['Nombre completo', 'CCT 1', 'LONGITUD 1', 'LATITUD 1', 'geometry 1', 'CCT 2', 'LONGITUD 2', 'LATITUD 2', 'geometry 2'])

filas = []

# Mostrar los DataFrames creados
for name, df_group in dataframes.items():
    #print('Esto es ', df_group)
    for (idx, row), (idy, rowy) in combinations(df_group.iterrows(), 2):
        punto = Point(row['LONGITUD'], row['LATITUD'])
        poligono = rowy['geometry']
        print(f"Escuela del punto: {row['CCT']}")
        print(f"Escuela del poligono: {rowy['CCT']}")
        dentro = punto.within(poligono)
        print(f"dentro = {dentro}")
        if not dentro:
            filappend({'Nombre completo': row['Nombre completo'],
                           'CCT 1': row['CCT'],
                           'LONGITUD 1': row['LONGITUD'],
                           'LATITUD 1': row['LATITUD'],
                           'geometry 1': row['geometry'],
                           'CCT 2': rowy['CCT'],
                           'LONGITUD 2': rowy['LONGITUD'],
                           'LATITUD 2': rowy['LATITUD'],
                           'geometry 2': rowy['geometry'],
                           }, ignore_index=True)

Escuela del punto: Maria Luisa Ross
Escuela del poligono: Independencia
dentro = False


AttributeError: 'DataFrame' object has no attribute 'append'